<a href="https://colab.research.google.com/github/JuWanMaeng/papaer-review/blob/main/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.layers import Input,Flatten,Dense,BatchNormalization,Conv2D,AvgPool2D,MaxPool2D,Activation,Add
from keras.models import Sequential,Model
from keras.layers import Concatenate
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [ ]:
(x_train,y_train),(x_test,y_test)=keras.datasets.cifar10.load_data()

In [ ]:
CLASS_NACLASS_NAMES= ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
print(x_train.shape)

(50000, 32, 32, 3)


In [ ]:
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=0.1)

train_ds=tf.data.Dataset.from_tensor_slices((x_train,y_train))
test_ds=tf.data.Dataset.from_tensor_slices((x_test,y_test))
val_ds=tf.data.Dataset.from_tensor_slices((x_val,y_val))

In [ ]:
def process_images(image,label):
  image=tf.image.resize(image,(224,224))
  return image,label

In [ ]:
train_ds=train_ds.batch(batch_size=128)
test_ds=test_ds.batch(batch_size=128)
val_ds=val_ds.batch(batch_size=128)

In [ ]:
train_ds=train_ds.map(process_images)
test_ds=test_ds.map(process_images)
val_ds=val_ds.map(process_images)

In [ ]:
def resnet_identity(x,filters):
  #차원이 변하지 않는 block
  
  #we will have 3 blocks and then input will be added
  x_skip=x
  f1,f2=filters

 # first block
  x=Conv2D(f1,(1,1),1,padding='valid',kernel_regularizer=keras.regularizers.l2(0.001))(x)
  x=BatchNormalization()(x)
  x=Activation(keras.activations.relu)(x)

  # second block
  x=Conv2D(f1,(3,3),1,padding='same',kernel_regularizer=keras.regularizers.l2(0.001))(x)
  x=BatchNormalization()(x)
  x=Activation(keras.activations.relu)(x)

  # third block
  x=Conv2D(f2,(1,1),1,padding='valid',kernel_regularizer=keras.regularizers.l2(0.001))(x)
  x=BatchNormalization()(x)

  x=Add()([x,x_skip])
  x=Activation(keras.activations.relu)(x)

  return x




In [ ]:
from keras.regularizers import L2
def resnet_conv(x,s,filters):
  # input 의 차원이 1x1 convolution과 stride=2에 의해 바뀌게 된다.
  # 그러므로 skip connection도 차원 변화를 처리해줘야 한다.

  x_skip=x
  f1,f2=filters

  # first block
  x=Conv2D(f1,(1,1),s,padding='valid',kernel_regularizer=keras.regularizers.l2(0.001))(x)
  x=BatchNormalization()(x)
  x=Activation(keras.activations.relu)(x)

  # second block
  x=Conv2D(f1,(3,3),1,padding='same',kernel_regularizer=keras.regularizers.l2(0.001))(x)
  x=BatchNormalization()(x)
  x=Activation(keras.activations.relu)(x)

  # thrid block
  x=Conv2D(f2,(1,1),1,padding='valid',kernel_regularizer=keras.regularizers.l2(0.001))(x)

  # shortcut
  x_skip=Conv2D(f2,(1,1),s,padding='valid',kernel_regularizer=keras.regularizers.l2(0.001))(x_skip)
  x_skip=BatchNormalization()(x_skip)

  # add
  x=Add()([x,x_skip])
  x=Activation(keras.activations.relu)(x)

  return x


  

In [ ]:
from keras.layers.convolutional import ZeroPadding2D
def resnet50():

  input_im=Input(shape=(224,224,3))
  x=ZeroPadding2D(padding=(3,3))(input_im)
  # 1st stage
  x=Conv2D(64,(7,7),2)(x)
  x=BatchNormalization()(x)
  x=Activation(keras.activations.relu)(x)
  x=MaxPool2D((3,3),2)(x)

  # 2nd stage
  
  x=resnet_conv(x,1,(64,256))
  x=resnet_identity(x,(64,256))
  x=resnet_identity(x,(64,256))

  # 3rd stage
  x=resnet_conv(x,2,(128,512))  
  x=resnet_identity(x,(128,512))
  x=resnet_identity(x,(128,512))
  x=resnet_identity(x,(128,512))

  #4th stage
  x=resnet_conv(x,2,(256,1024))
  x=resnet_identity(x,(256,1024))
  x=resnet_identity(x,(256,1024))
  x=resnet_identity(x,(256,1024))
  x=resnet_identity(x,(256,1024))
  x=resnet_identity(x,(256,1024))

  # 5th stage
  x=resnet_conv(x,2,(512,2048))
  x=resnet_identity(x,(512,2048))
  x=resnet_identity(x,(512,2048))

  x=AvgPool2D((7,7))(x)

  x=Flatten()(x)
  x=Dense(1000,activation='softmax',kernel_initializer='he_normal')(x)

  model=Model(inputs=input_im,outputs=x,name='Resnet50')

  return model


  

In [ ]:
a=resnet50()
a.summary()

Model: "Resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_5 (ZeroPadding2D (None, 230, 230, 3)  0           input_9[0][0]                    
__________________________________________________________________________________________________
conv2d_327 (Conv2D)             (None, 112, 112, 64) 9472        zero_padding2d_5[0][0]           
__________________________________________________________________________________________________
batch_normalization_302 (BatchN (None, 112, 112, 64) 256         conv2d_327[0][0]                 
___________________________________________________________________________________________